In [1]:
# ============================================
#  Skincare Compatibility API (Flask + Ngrok)
# ============================================

!pip install flask flask-ngrok scikit-learn joblib pandas -q

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np
import pandas as pd
from difflib import get_close_matches
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
#  Load model dan dataset
# -----------------------------
model = joblib.load("skincare_model.pkl")
df = pd.read_csv("unified_cleaned_products.csv", encoding="utf-8", engine="python")

# -----------------------------
#  Definisikan fungsi bantu
# -----------------------------
def find_closest_product_name(name, df, cutoff=0.3):
    """Cari nama produk paling mirip dari dataset (fuzzy matching)."""
    all_names = df["product_name"].tolist()
    matches = get_close_matches(name, all_names, n=5, cutoff=cutoff)
    return matches

# -----------------------------
#  Inisialisasi Flask app
# -----------------------------
app = Flask(__name__)
run_with_ngrok(app)  # supaya dapat URL publik

@app.route("/")
def home():
    return "🧴 Skincare Compatibility API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    product1 = data.get("product1")
    product2 = data.get("product2")

    if not product1 or not product2:
        return jsonify({"status": "error", "message": "Harap masukkan dua nama produk."})

    # Cari nama produk paling mirip
    matches1 = find_closest_product_name(product1, df)
    matches2 = find_closest_product_name(product2, df)

    if not matches1 or not matches2:
        return jsonify({
            "status": "error",
            "message": "❌ Salah satu produk tidak ditemukan.",
            "suggestions1": matches1 or [],
            "suggestions2": matches2 or []
        })

    name1, name2 = matches1[0], matches2[0]
    p1 = df[df["product_name"] == name1].iloc[0]
    p2 = df[df["product_name"] == name2].iloc[0]

    ing1, ing2 = p1["parsed_ingredients"], p2["parsed_ingredients"]

    # --- Buat fitur ---
    len_diff = abs(len(ing1) - len(ing2))
    shared = len(set(ing1) & set(ing2))
    jaccard = len(set(ing1) & set(ing2)) / len(set(ing1) | set(ing2)) if len(set(ing1) | set(ing2)) > 0 else 0

    vec = CountVectorizer().fit([" ".join(ing1), " ".join(ing2)])
    tf1 = vec.transform([" ".join(ing1)])
    tf2 = vec.transform([" ".join(ing2)])
    cosine_sim = cosine_similarity(tf1, tf2)[0][0]

    X_new = np.array([[len_diff, shared, jaccard, cosine_sim]])
    pred = model.predict(X_new)[0]

    # Ambil confidence dengan aman
    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X_new)[0]
        prob = probs[pred] if probs.size > 1 else 1.0
    else:
        prob = 1.0

    result = "✅ Cocok digunakan bersama" if pred == 1 else "⚠️ Tidak disarankan dipakai bersama"

    return jsonify({
        "status": "ok",
        "product1": name1,
        "product2": name2,
        "result": result,
        "confidence": round(float(prob), 2)
    })

# -----------------------------
#  Jalankan API
# -----------------------------
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            